<a href="https://colab.research.google.com/github/SuperPanww/courses_ML20/blob/master/hw2_panww.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id '1KSFIRh0-_Vr7SdiSCZP1ItV7bXPxMD92' --output data.tar.gz
!tar -zxvf data.tar.gz
!ls

Downloading...
From: https://drive.google.com/uc?id=1KSFIRh0-_Vr7SdiSCZP1ItV7bXPxMD92
To: /content/data.tar.gz
6.11MB [00:00, 95.1MB/s]
data/
data/sample_submission.csv
data/test_no_label.csv
data/train.csv
data/X_test
data/X_train
data/Y_train
data  data.tar.gz  sample_data


In [0]:
import numpy as np
np.random.seed(0)
X_train_fpath = './data/X_train'
Y_train_fpath = './data/Y_train'
X_test_fpath = './data/X_test'
output_fpath = './output_{}.csv'

with open(X_train_fpath) as f:
  next(f)
  X_train = np.array([line.strip('\n').split(',')[1:] for line in f],dtype = float)
with open(Y_train_fpath) as f:
  next(f)
  Y_train = np.array([line.strip('\n').split(',')[1:] for line in f],dtype = float)
with open(X_test_fpath) as f:
  next(f)
  X_test = np.array([line.strip('\n').split(',')[1:] for line in f],dtype = float)

In [0]:
def _normalize(X,train = True,specified_colum = None,X_mean = None,X_std = None):
  if specified_colum == None:
    specified_colum = np.arange(X.shape[1])
  if train:
    X_mean = np.mean(X[:,specified_colum],0).reshape(1,-1)
    X_std = np.std(X[:,specified_colum],0).reshape(1,-1)
  X[:,specified_colum] = (X[:,specified_colum]-X_mean) / (X_std + 1e-8)
  return X,X_mean,X_std

In [0]:
def _train_dev_split(X,Y,dev_ratio = 0.25):
  train_size = int(int(len(X)*(1-dev_ratio)))
  return X[:train_size],Y[:train_size],X[train_size:],Y[train_size:]

In [9]:
X_train,X_mean,X_std = _normalize(X_train,train=True)
X_test,_,_ = _normalize(X_test,train=False,specified_colum=None,X_mean=X_mean,X_std=X_std)
dev_ratio = 0.1
X_train,Y_train,X_dev,Y_dev = _train_dev_split(X_train,Y_train,dev_ratio=dev_ratio)

train_size = X_train.shape[0]
dev_size = X_dev.shape[0]
test_size = X_test.shape[0]
data_dim = X_train.shape[1]

print('Size of training set: {}'.format(train_size))
print('Size of development set: {}'.format(dev_size))
print('Size of testing set: {}'.format(test_size))
print('Dimension of data: {}'.format(data_dim))

Size of training set: 48830
Size of development set: 5426
Size of testing set: 27622
Dimension of data: 510


In [0]:
def _shuffle(X,Y):
  randomize = np.arange(len(X))
  np.random.shuffle(randomize)
  return (X[randomize],Y[randomize])

In [0]:
def _sigmoid(z):
  return np.clip(1/(1.0 + np.exp(-z)),1e-8,1-(1e-8))

In [0]:
def _f(X,w,b):
  return _sigmoid(np.matmul(X,w)+b)

In [0]:
def _predict(X,w,b):
  return np.round(_f(X,w,b)).astype(np.int)

In [0]:
def _accuracy(Y_pred,Y_label):
  acc = 1 - np.mean(np.abs(Y_pred - Y_label))
  return acc

In [0]:
def _cross_entropy_loss(y_pred,Y_label):
  